# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
)

API key looks good so far


In [13]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

SYSTEM_PROMPT = "You are a helpful coding assistant. Explain technical concepts clearly and concisely. Include code examples. Use markdown for formatting."

In [14]:
# Get gpt-4o-mini to answer, with streaming
def get_authors():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

get_authors()

The code snippet you provided uses the `yield from` statement in Python, combined with a set comprehension. Here’s a breakdown of what it does and why:

### Breakdown of the Code

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This part of the code creates a set of authors from a list (or any iterable) called `books`.
   - It iterates over each `book` in `books`.
   - For each `book`, it tries to get the value associated with the key `"author"` using `book.get("author")`. 
   - The `if book.get("author")` condition ensures that only books that actually have an author (`author` is not `None` or an empty string) are considered.
   - The result is a set containing unique authors from the `books` iterable.

2. **`yield from`**: 
   ```python
   yield from
   ```
   - `yield from` is used in generator functions to yield all values from another iterable. This allows a generator to delegate part of its operations to another generator or iterable.
   - In this case, it yields each author from the set of authors generated by the set comprehension.

### Why Use This Code?

- **Eliminates Duplicates**: By using a set comprehension, the code automatically filters out duplicate authors. This is useful when you want a unique collection of authors without manually checking for duplicates.
- **Efficient Iteration**: The `yield from` expression allows the function to yield each author sequentially, which can be memory efficient for large datasets since it does not require storing all authors in a list beforehand.
- **Readable and Concise**: This approach is clearer and more concise than using explicit loops to filter and yield authors.

### Example

Here's an example to illustrate how this might be used in practice:

```python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example data
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": "Author 1"},  # Duplicate author
    {"title": "Book D"},  # No author
    {"title": "Book E", "author": "Author 3"},
]

# Using the generator
for author in get_unique_authors(books):
    print(author)
```

### Output

```
Author 1
Author 2
Author 3
```

In this example, the generator yields each unique author found in the `books` collection, demonstrating the functionality of the provided code.

In [15]:
# Get Llama 3.2 to answer
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

def get_authors_with_llama():
    stream = ollama_client.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

get_authors_with_llama()


**Code Breakdown**
===============

The given code snippet uses the `yield from` syntax to generate a sequence of values.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what it does:

*   **Dictionary comprehension**: `{book.get("author") for book in books if book.get("author")}` is a dictionary comprehension. It creates a new dictionary containing only the author names.
    *   `for book in books`: Iterates over each item (`book`) in the list or iterable `books`.
    *   `if book.get("author")`: Filters the items to include only those that have an "author" key with a value of some type (e.g., string).
    *   `book.get("author")`: Retrieves the author name from each valid item.
*   **`yield from`**: This syntax is used to delegate iteration to another iterable.
    *   It takes any iterable (like a dictionary comprehension) and yields its values directly, without having to store them in memory.

**Why `yield from`?**
-------------------

You might wonder why use `yield from` instead of simply iterating over the dictionary comprehension. The main advantage is that `yield from` can significantly reduce memory usage for large datasets or infinite sequences.

By using `yield from`, we only yield one value at a time, allowing us to generate the sequence without storing all values in memory. This makes the code more memory-efficient and suitable for large inputs.

**Example Usage**
----------------

```python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": "Jane Smith"},
    {"title": "Book 3"}
]

for author in yield from {book["author"] for book in books if "author" in book}:
    print(author)
```

This will output:

```
John Doe
Jane Smith
```